# Use LabelKit to experiment and evaluate different approaches
There are many ways to build a labeling pipeline that all will accomplish the same result. The goal of `LabelKit` is to empower rapid and robust experimentation so that you can understand the performance, accuracy, and cost tradeoffs between approaches.

In this example, we'll experiment with a few different approaches to a categorization pipeline we want to build. `LabelKit` will make this experimentation quick and at the end we'll have a solid understanding of how different approaches perform. 


### Task
The task at hand is to categorize furniture items into a multi-level taxonomy based on their name and description. 

For example
Name: `Blair Table by homestyles`

Description: `This Blair Table by homestyles is perfect for Sunday brunches or game night. The round pedestal table is available as shown, or as part of a five-piece set. Features solid hardwood construction in a black finish that can easily match a traditional or contemporary aesthetic. Measures: 30"H x 42" Diameter`

Correct classification: `Tables & Desks > Dining Tables`

### Approaches
There are two different approaches we want to try.
1. LLMs + Embedding 
2. Heiarchical prompting


In [270]:
import os

# Set your OpenAI key here or set it as an environment variable
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

In [267]:
# %pip install cohere

import pandas as pd
from labelkit import *
from pydantic import BaseModel, Field
import cohere
import os
import numpy as np
from typing import List

## Data processing
We'll start out with reading in our data and building our taxonomy. The process of building a taxonomy is a project in and of itself. There are also many taxonomies available online that you can use. In our case, we're building our taxonomy based on our ground truth dataset. Since we have such a large dataset we can be reasonably confident that all values are represented. As you'll see our approach does not use the ground truth data as training data so it will be easy for us to expand the taxonomy without needing additional data. 

In [43]:
df = pd.read_csv('./furniture_clean.csv')

In [44]:
# Remove the 'Furniture > ' from each string in the 'category' column since they all start with Furniture.
df['category_new'] = df['category'].str.replace('Furniture > ', '')


For our embeddings approach we want the taxonomy to be a single string. We'll create the taxonomy from the ground truth data. 

In [269]:
taxonomy = list(set(df['category_new']))
taxonomy[0:5]


['Chairs > Recliners',
 'Mattresses & Box Springs > Box Springs & Foundations',
 'Sets > Dining Room Furniture Sets',
 'Storage > Bathroom Vanities',
 'Outdoor Seating > Outdoor Chairs']

However, for our heiarchical approach we need to understand the taxonomy a little more so we'll create a lookup table between first and second level categories.

In [46]:
# Create a lookup table with first level taxonomy as keys and second level as values
lookup_table = df['category_new'].str.split(' > ', expand=True).groupby(0)[1].apply(list).apply(set)
lookup_table['Chairs']

{'Accent Chairs', 'Desk Chairs', 'Dining Chairs', 'Recliners'}

## Building our pipeline using LabelKit

### Approach 1: Embeddings
The first approach is similar to the approach we took in the `Product Categorization` example we gave in the project repo. We are omitting the Google Search step because we already have item descriptions. 
1. Write a simple description of the product given name and description
2. Vector embedding search for top N categories
3. LLM: pick the best category



In [271]:
short_description_prompt = lambda row: f"""
You are given a product name and description for a piece of furniture.
Return a single sentence decribing the product.
Product name: {row['name']}
Product description: {row['description']}
"""

class ShortDescription(BaseModel):
  short_description: str = Field(description="A single sentence describing the product")
  
short_description_step = steps.LLMStep(
  prompt=short_description_prompt,
  model=models.gpt35,
  out_schema=ShortDescription,
  name="short_description"
)

We are using Cohere to embed both our description and the taxonomy but you can substitute in any embeddings provider with the `EmbeddingClassificationStep`. Unlike LLMs that are good at ignoring irrelevent information, we've learned from experience that short, simple descriptions work better in embedding space than trying to include too much. This is something you can and should experiment with. 

In [167]:
# set your cohere api key as an env var or set it directly here
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')
co = cohere.Client(COHERE_API_KEY)

def embed(texts: List[str]):
  embeddings = co.embed(
    model="embed-english-v3.0",
    texts=texts,
    input_type='classification'
  ).embeddings
  return np.array(embeddings).astype('float32')

embedding_search_prompt = lambda row: row["short_description"]

embedding_search_step = steps.EmbeddingClassificationStep(
  search_prompt= embedding_search_prompt,
  embed=embed,
  k=5,    
  categories=taxonomy,
  name="embedding_search"
)

We now take the result of the embeddings and ask the LLM to pick the best response. It's important that our embedding search is optimized for recall because if the correct answer doesn't exist in the response our categorize step will have no chance of succeeding. 

In [168]:

def categorize_prompt(row):
    categories = ""
    i = 1
    while f"category{i}" in row:
        categories += f'{i}. {row[f"category{i}"]}\n'
        i += 1

    return f"""
    You are given a product description and {i-1} options for the product's category.
    Pick the index of the most accurate category.
    The index must be between 1 and {i-1}.
    Product description: {row['short_description']}
    Categories:
    {categories}
    """
    
class CategoryIndex(BaseModel):
    category_index: int = Field(description="The index of the most accurate category")
    
categorize_step = steps.LLMStep(
  prompt=categorize_prompt,
  model=models.gpt35,
  out_schema=CategoryIndex,
  name="categorize"
)

By returning just the index we can ensure that the actual string we use is in the taxonomy since LLMs sometimes hallucinate characters. Additionally, we don't need to waste response tokens on printing the entire string.

In [169]:
class Category(BaseModel):
    predicted_category: str = Field(description="The most accurate category")

select_category_step = steps.CustomStep(
  transform=lambda row: {"predicted_category": row[f'category{row["category_index"]}']},
  out_schema=Category,
  name="select_category"
)

We'd like to test our end to end pipeline to make sure it works before we go any further. We'll make a copy of the first five rows of the dataframe and run the pipeline to make sure it works

In [273]:
test_df = df.head(5).copy()

In [274]:
evaluate = lambda row: row['predicted_category'].lower() == row['category_new'].lower()

categorizer = pipeline.Pipeline([
  short_description_step, 
  embedding_search_step, 
  categorize_step,
  select_category_step
], evaluation_fn=evaluate)

categorizer.apply(test_df)

,name,description,category,brand.name,category_new,__short_description__,short_description,category3,category4,category5,category1,category2,__categorize__,category_index,predicted_category
0,EnGauge Deluxe Bedframe,Introducing the Engauge Deluxe Bedframe - the ...,Furniture > Beds & Headboards > Bedframes,NaN,Beds & Headboards > Bedframes,"{'input_tokens': 313, 'output_tokens': 56, 'su...",Introducing the Engauge Deluxe Bedframe - the ...,Beds & Headboards > Headboards,Mattresses & Box Springs > Mattresses,Mattresses & Box Springs > Box Springs & Found...,Beds & Headboards > Bedframes,Beds & Headboards > Beds,"{'input_tokens': 202, 'output_tokens': 10, 'su...",1,Beds & Headboards > Bedframes
1,Sparrow & Wren Sullivan King Channel-Stitched ...,"85""L x 83""W x 56""H | Total weight: 150 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 169, 'output_tokens': 51, 'su...",The Sparrow & Wren Sullivan King Channel-Stitc...,Mattresses & Box Springs > Mattresses,Beds & Headboards > Bedframes,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Headboards,Beds & Headboards > Beds,"{'input_tokens': 196, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds
2,Queen Bed With Frame,Dimensions:Head Board -49H x 63.75W x 1.5DFoot...,Furniture > Beds & Headboards > Beds,Hillsdale,Beds & Headboards > Beds,"{'input_tokens': 124, 'output_tokens': 56, 'su...",The Queen Bed With Frame features a head board...,Beds & Headboards > Headboards,Kids Beds & Headboards > Kid's Beds,Sets > Bedroom Furniture Sets,Beds & Headboards > Bedframes,Beds & Headboards > Beds,"{'input_tokens': 198, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds
3,Dylan Queen Bed,Add a touch of a modern farmhouse to your bedr...,Furniture > Beds & Headboards > Beds,NaN,Beds & Headboards > Beds,"{'input_tokens': 140, 'output_tokens': 41, 'su...",Add a touch of modern farmhouse charm to your ...,Beds & Headboards > Bedframes,Sets > Bedroom Furniture Sets,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Beds,Beds & Headboards > Headboards,"{'input_tokens': 183, 'output_tokens': 10, 'su...",1,Beds & Headboards > Beds
4,Sparrow & Wren Mara Full Diamond-Tufted Bed,"78""L x 56""W x 51""H | Total weight: 130 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 168, 'output_tokens': 71, 'su...",The Sparrow & Wren Mara Full Diamond-Tufted Be...,Beds & Headboards > Bedframes,Mattresses & Box Springs > Mattresses,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Headboards,Beds & Headboards > Beds,"{'input_tokens': 215, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds


Let's create a table to nicely visualize our pipeline statistics

In [278]:
from prettytable import PrettyTable
import json

def create_pretty_table(statistics_json_list, names=None):
    # If no names are provided, generate default names
    if not names or len(names) != len(statistics_json_list):
        names = [f"Pipeline {i+1}" for i in range(len(statistics_json_list))]
    
    # Create a PrettyTable object
    table = PrettyTable()
    
    # Add columns to the table
    table.field_names = ["Name", "Model", "Input Tokens", "Output Tokens", "Num Success", "Num Failure", "Total Latency"]
    
    # Iterate over each statistics JSON and its corresponding name
    for name, statistics_json in zip(names, statistics_json_list):
        # Parse the JSON statistics data
        data = json.loads(statistics_json)
        
        # Extract shared statistics
        num_success = data.get("num_success", "")
        num_failure = data.get("num_failure", "")
        total_latency = data.get("total_latency", "")
        
        # Initialize a flag to indicate the first row for shared statistics
        first_row = True
        
        # Add rows to the table for each model
        for model, input_tokens in data.get("input_tokens", {}).items():
            output_tokens = data.get("output_tokens", {}).get(model, "")
            
            # Only add shared statistics to the first row
            if first_row:
                table.add_row([name, model, input_tokens, output_tokens, num_success, num_failure, total_latency])
                first_row = False
            else:
                table.add_row(["", model, input_tokens, output_tokens, "", "", ""])
    
    # Return the table as a string
    return table

In [280]:
# Example usage with the provided data from the model dumps
statistics_json_list = [
    categorizer.statistics.model_dump_json(),
]

names = ["Categorizer"]

# Create and print the pretty table
pretty_table = create_pretty_table(statistics_json_list, names)
print(pretty_table)

+-------------+--------------------+--------------+---------------+-------------+-------------+--------------------+
|     Name    |       Model        | Input Tokens | Output Tokens | Num Success | Num Failure |   Total Latency    |
+-------------+--------------------+--------------+---------------+-------------+-------------+--------------------+
| Categorizer | gpt-3.5-turbo-0125 |     2948     |      375      |      5      |      0      | 13.613645457051462 |
+-------------+--------------------+--------------+---------------+-------------+-------------+--------------------+


In [281]:
print(f"Accuracy: {categorizer.score}")

Accuracy: 1.0


Our pipeline is doing well but that's only on 5 data points. Let's try it on a few more.

In [282]:
test_df100 = df.head(100).copy()
categorizer.apply(test_df100)

,name,description,category,brand.name,category_new,__short_description__,short_description,category3,category4,category5,category1,category2,__categorize__,category_index,predicted_category
0,EnGauge Deluxe Bedframe,Introducing the Engauge Deluxe Bedframe - the ...,Furniture > Beds & Headboards > Bedframes,NaN,Beds & Headboards > Bedframes,"{'input_tokens': 313, 'output_tokens': 71, 'su...",The EnGauge Deluxe Bedframe is the ultimate so...,Mattresses & Box Springs > Box Springs & Found...,Beds & Headboards > Beds,Beds & Headboards > Headboards,Beds & Headboards > Bedframes,Mattresses & Box Springs > Mattresses,"{'input_tokens': 217, 'output_tokens': 10, 'su...",1,Beds & Headboards > Bedframes
1,Sparrow & Wren Sullivan King Channel-Stitched ...,"85""L x 83""W x 56""H | Total weight: 150 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 169, 'output_tokens': 91, 'su...",The Sparrow & Wren Sullivan King Channel-Stitc...,Beds & Headboards > Bedframes,Kids Beds & Headboards > Kid's Beds,Sets > Bedroom Furniture Sets,Beds & Headboards > Headboards,Beds & Headboards > Beds,"{'input_tokens': 233, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds
2,Queen Bed With Frame,Dimensions:Head Board -49H x 63.75W x 1.5DFoot...,Furniture > Beds & Headboards > Beds,Hillsdale,Beds & Headboards > Beds,"{'input_tokens': 124, 'output_tokens': 64, 'su...",The Queen Bed With Frame is a stylish and spac...,Beds & Headboards > Beds,Kids Beds & Headboards > Kid's Beds,Sets > Bedroom Furniture Sets,Beds & Headboards > Bedframes,Beds & Headboards > Headboards,"{'input_tokens': 206, 'output_tokens': 10, 'su...",1,Beds & Headboards > Bedframes
3,Dylan Queen Bed,Add a touch of a modern farmhouse to your bedr...,Furniture > Beds & Headboards > Beds,NaN,Beds & Headboards > Beds,"{'input_tokens': 140, 'output_tokens': 42, 'su...",The Dylan Queen Bed combines modern farmhouse ...,Beds & Headboards > Bedframes,Sets > Bedroom Furniture Sets,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Beds,Beds & Headboards > Headboards,"{'input_tokens': 184, 'output_tokens': 10, 'su...",1,Beds & Headboards > Beds
4,Sparrow & Wren Mara Full Diamond-Tufted Bed,"78""L x 56""W x 51""H | Total weight: 130 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 168, 'output_tokens': 62, 'su...",The Sparrow & Wren Mara Full Diamond-Tufted Be...,Mattresses & Box Springs > Mattresses,Beds & Headboards > Bedframes,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Headboards,Beds & Headboards > Beds,"{'input_tokens': 207, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Modway Melanie Tufted Button Upholstered Fabri...,"Twin | Clean lines, a straightforward profile,...",Furniture > Beds & Headboards > Beds,Modway,Beds & Headboards > Beds,"{'input_tokens': 225, 'output_tokens': 58, 'su...",The Modway Melanie Tufted Button Upholstered F...,Sets > Bedroom Furniture Sets,Beds & Headboards > Bedframes,Kids Beds & Headboards > Kid's Beds,Beds & Headboards > Beds,Beds & Headboards > Headboards,"{'input_tokens': 200, 'output_tokens': 10, 'su...",1,Beds & Headboards > Beds
96,Concord Queen Panel Bed,Looking for a new bed that has it all? Check o...,Furniture > Beds & Headboards > Beds,Daniel's Amish,Beds & Headboards > Beds,"{'input_tokens': 205, 'output_tokens': 45, 'su...",The Concord Queen Panel Bed is a contemporary ...,Beds & Headboards > Bedframes,Kids Beds & Headboards > Kid's Beds,Sets > Bedroom Furniture Sets,Beds & Headboards > Headboards,Beds & Headboards > Beds,"{'input_tokens': 187, 'output_tokens': 10, 'su...",2,Beds & Headboards > Beds
97,Sparrow & Wren Myers King Bed,"Dimensions: 85""L x 82""W x 56""H | Headboard hei...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 271, 'output_tokens': 72, 'su...",The Sparrow & Wren Myers King Bed is 

In [283]:
statistics_json_list = [
    categorizer.statistics.model_dump_json(),
]

names = ["Categorizer"]

# Create and print the pretty table
pretty_table = create_pretty_table(statistics_json_list, names)
print(pretty_table)
print(f"Accuracy: {categorizer.score}")

+-------------+--------------------+--------------+---------------+-------------+-------------+-------------------+
|     Name    |       Model        | Input Tokens | Output Tokens | Num Success | Num Failure |   Total Latency   |
+-------------+--------------------+--------------+---------------+-------------+-------------+-------------------+
| Categorizer | gpt-3.5-turbo-0125 |    40841     |      6835     |     100     |      0      | 450.6441533318721 |
+-------------+--------------------+--------------+---------------+-------------+-------------+-------------------+
Accuracy: 0.93


At current gpt-3.5-turbo pricing this batch of 100 requests cost about $0.030673 and took seven minutes and thirty seconds for 93% accuracy. Let's see how heiarchical prompting does. 

### Approach 2: Heiarchical prompting
Next we want to try forgoing embeddings all together and simply stuffing all of the categories into the prompt. There are too many categories to do this all in one go but we can use the fact that our categories are heiarchical and take a step by step approach.
1. LLM: given product name, description, and first level categories, pick the best one.
2. LLM: given product name, description, and second level categories, pick the best one.

We may want to iterate a bit on this process. For example, we may want to use one model in step 1 and a different model in step 2. `LabelKit` makes this type of hyperparameter tuning easy and robust.

In our first step we're just asking the model to pick the right top level category. This is a relatively easy task if the categories are non-overlapping or can be very difficult if there are multiple correct answers. We'll only know by trying and inspecting our losses.

In [286]:
first_level_categories = list(lookup_table.keys())

def first_level_category_prompt(row):
    i = len(first_level_categories)

    return f"""
    You are given a product name, description and {i} options for the product's top level category.
    Pick the index of the most accurate category.
    The index must be between 1 and {i}.
    Product description: {row['description']}
    Product name: {row['name']}
    Categories:
    {first_level_categories}
    """
    
class FirstLevelCategoryIndex(BaseModel):
    first_category_index: int = Field(description="The index of the most accurate first level category")
    
first_level_category_step = steps.LLMStep(
  prompt=first_level_category_prompt,
  model=models.gpt35,
  out_schema=FirstLevelCategoryIndex,
  name="first_categorize"
)

In [287]:
class FirstCategory(BaseModel):
    predicted_first_category: str = Field(description="The most accurate first level category")

select_first_category_step = steps.CustomStep(
  transform=lambda row: {"predicted_first_category": first_level_categories[row["first_category_index"] - 1]},
  out_schema=FirstCategory,
  name="select_first_category"
)

Next we'll give the second layer of the taxonomy to the model to classify. Just as before are trying to predict the index to make sure our final output is valid. 

In [288]:
def second_level_category_prompt(row):
    second_level_categories = list(lookup_table[row['predicted_first_category']])
    i = len(second_level_categories)

    return f"""
    You are given a product name, description, first level category 
    and {i} options for the product's second level category.
    Pick the index of the most accurate category.
    The index must be between 1 and {i}.
    Product description: {row['description']}
    Product name: {row['name']}
    First level category: {row['predicted_first_category']}
    Categories:
    {second_level_categories}
    """
    
class SecondLevelCategoryIndex(BaseModel):
    second_category_index: int = Field(description="The index of the most accurate second level category")
    
second_level_category_step = steps.LLMStep(
  prompt=second_level_category_prompt,
  model=models.gpt35,
  out_schema=SecondLevelCategoryIndex,
  name="second_categorize"
)

In [289]:
class SecondCategory(BaseModel):
    predicted_second_category: str = Field(description="The most accurate second level category")

select_second_category_step = steps.CustomStep(
  transform=lambda row: {"predicted_second_category": list(lookup_table[row['predicted_first_category']])[row["second_category_index"] - 1]},
  out_schema=SecondCategory,
  name="select_second_category"
)

Let's combine our results so we can properly compare to our ground truth column. 

In [290]:
class PredictedTaxonomy(BaseModel):
    predicted_taxonomy: str = Field(description="The predicted taxonomy based on LLM categorization")

combine_taxonomy_step = steps.CustomStep(
    transform=lambda row: {"predicted_taxonomy": f"{row['predicted_first_category']} > {row['predicted_second_category']}"},
    out_schema=PredictedTaxonomy,
    name='combine_taxonomy'
)

In [291]:
test_df2 = df.head(5).copy()

evaluate2 = lambda row: row['predicted_taxonomy'].lower() == row['category_new'].lower()

categorizer_llm = pipeline.Pipeline([
  first_level_category_step, 
  select_first_category_step,
  second_level_category_step,
  select_second_category_step,
  combine_taxonomy_step
], evaluation_fn=evaluate2)

categorizer_llm.apply(test_df2)

,name,description,category,brand.name,category_new,__first_categorize__,first_category_index,predicted_first_category,__second_categorize__,second_category_index,predicted_second_category,predicted_taxonomy
0,EnGauge Deluxe Bedframe,Introducing the Engauge Deluxe Bedframe - the ...,Furniture > Beds & Headboards > Bedframes,NaN,Beds & Headboards > Bedframes,"{'input_tokens': 419, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 372, 'output_tokens': 11, 'su...",1,Bedframes,Beds & Headboards > Bedframes
1,Sparrow & Wren Sullivan King Channel-Stitched ...,"85""L x 83""W x 56""H | Total weight: 150 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 275, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 228, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
2,Queen Bed With Frame,Dimensions:Head Board -49H x 63.75W x 1.5DFoot...,Furniture > Beds & Headboards > Beds,Hillsdale,Beds & Headboards > Beds,"{'input_tokens': 230, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 183, 'output_tokens': 11, 'su...",1,Bedframes,Beds & Headboards > Bedframes
3,Dylan Queen Bed,Add a touch of a modern farmhouse to your bedr...,Furniture > Beds & Headboards > Beds,NaN,Beds & Headboards > Beds,"{'input_tokens': 246, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 199, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
4,Sparrow & Wren Mara Full Diamond-Tufted Bed,"78""L x 56""W x 51""H | Total weight: 130 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 274, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 227, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds


It works, let's run it on some more data like we did before. 

In [294]:
test_df2_100 = df.head(100).copy()
categorizer_llm.apply(test_df2_100)

,name,description,category,brand.name,category_new,__first_categorize__,first_category_index,predicted_first_category,__second_categorize__,second_category_index,predicted_second_category,predicted_taxonomy
0,EnGauge Deluxe Bedframe,Introducing the Engauge Deluxe Bedframe - the ...,Furniture > Beds & Headboards > Bedframes,NaN,Beds & Headboards > Bedframes,"{'input_tokens': 419, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 372, 'output_tokens': 11, 'su...",1,Bedframes,Beds & Headboards > Bedframes
1,Sparrow & Wren Sullivan King Channel-Stitched ...,"85""L x 83""W x 56""H | Total weight: 150 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 275, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 228, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
2,Queen Bed With Frame,Dimensions:Head Board -49H x 63.75W x 1.5DFoot...,Furniture > Beds & Headboards > Beds,Hillsdale,Beds & Headboards > Beds,"{'input_tokens': 230, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 183, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
3,Dylan Queen Bed,Add a touch of a modern farmhouse to your bedr...,Furniture > Beds & Headboards > Beds,NaN,Beds & Headboards > Beds,"{'input_tokens': 246, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 199, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
4,Sparrow & Wren Mara Full Diamond-Tufted Bed,"78""L x 56""W x 51""H | Total weight: 130 lbs. | ...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 274, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 227, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Modway Melanie Tufted Button Upholstered Fabri...,"Twin | Clean lines, a straightforward profile,...",Furniture > Beds & Headboards > Beds,Modway,Beds & Headboards > Beds,"{'input_tokens': 331, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 284, 'output_tokens': 11, 'su...",1,Bedframes,Beds & Headboards > Bedframes
96,Concord Queen Panel Bed,Looking for a new bed that has it all? Check o...,Furniture > Beds & Headboards > Beds,Daniel's Amish,Beds & Headboards > Beds,"{'input_tokens': 311, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 264, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
97,Sparrow & Wren Myers King Bed,"Dimensions: 85""L x 82""W x 56""H | Headboard hei...",Furniture > Beds & Headboards > Beds,Sparrow & Wren,Beds & Headboards > Beds,"{'input_tokens': 377, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 330, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds
98,Loden Beige 3 Pc Queen Upholstered Bed with 2 ...,A classic design and sophisticated silhouette ...,Furniture > Beds & Headboards > Beds,Rooms To Go,Beds & Headboards > Beds,"{'input_tokens': 287, 'output_tokens': 11, 'su...",1,Beds & Headboards,"{'input_tokens': 240, 'output_tokens': 11, 'su...",3,Beds,Beds & Headboards > Beds


Let's compare approach 1 to approach 2. 

In [295]:
statistics_json_list = [
    categorizer.statistics.model_dump_json(),
    categorizer_llm.statistics.model_dump_json(),
]

names = ["Embeddings+LLM", "Heiarchical prompting"]

# Create and print the pretty table
pretty_table = create_pretty_table(statistics_json_list, names)
print(pretty_table)
print(f"Embeddings+LLM accuracy: {categorizer.score}")
print(f"Heiarchical prompting accuracy: {categorizer_llm.score}")

+-----------------------+--------------------+--------------+---------------+-------------+-------------+--------------------+
|          Name         |       Model        | Input Tokens | Output Tokens | Num Success | Num Failure |   Total Latency    |
+-----------------------+--------------------+--------------+---------------+-------------+-------------+--------------------+
|     Embeddings+LLM    | gpt-3.5-turbo-0125 |    40841     |      6835     |     100     |      0      | 450.6441533318721  |
| Heiarchical prompting | gpt-3.5-turbo-0125 |    58359     |      2423     |     100     |      0      | 227.19441783780348 |
+-----------------------+--------------------+--------------+---------------+-------------+-------------+--------------------+
Embeddings+LLM accuracy: 0.93
Heiarchical prompting accuracy: 0.83


Our heiarchical approach cost almost the same at $0.032814 / 100 rows. It was much faster but at the cost of accuracy. However, we're not done just yet. The power of `LabelKit` is that we can easily try many different permuations of our pipeline using a grid search.

## Grid search

Our first pipeline has three steps we want to search over.
1. Short description: vary the model
2. Embedding search: vary the number of results
3. Categorize: vary the model

It's not clear which permutation will work the best so we'll try all of them.

In [318]:
from labelkit.grid_search import GridSearch

params_grid = {
    'short_description_step': {
        'model': [models.gpt35, models.gpt4], 
    },
    'embedding_search_step': {
        'k': [3, 5],  
    },
    'categorize_step': {
        'model': [models.gpt35, models.gpt4], 
    },
}

small_df = df.head(20).copy()

grid_search = GridSearch(categorizer, params_grid)

results_df = grid_search.apply(small_df)

Iteration 1 of 8
Params:  {'short_description_step': {'model': 'gpt-3.5-turbo-0125'}, 'embedding_search_step': {'k': 3}, 'categorize_step': {'model': 'gpt-3.5-turbo-0125'}}
Result:  {'short_description_step__model': 'gpt-3.5-turbo-0125', 'embedding_search_step__k': 3, 'categorize_step__model': 'gpt-3.5-turbo-0125', 'score': 0.85, 'input_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 7963}), 'output_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 1366}), 'num_success': 20, 'num_failure': 0, 'total_latency': 43.87839541604626, 'index': 101600679592207735}
Iteration 2 of 8
Params:  {'short_description_step': {'model': 'gpt-3.5-turbo-0125'}, 'embedding_search_step': {'k': 3}, 'categorize_step': {'model': 'gpt-4-turbo-preview'}}


The results of our grid search are conveniently put into a dataframe for us to review

In [301]:
results_df

,short_description_step__model,embedding_search_step__k,score,input_tokens,output_tokens,num_success,num_failure,total_latency,index
0,gpt-3.5-turbo-0125,3,0.8,{'gpt-3.5-turbo-0125': 1934},{'gpt-3.5-turbo-0125': 349},5,0,9.993863,7476231247988649220
1,gpt-3.5-turbo-0125,5,1.0,{'gpt-3.5-turbo-0125': 1931},{'gpt-3.5-turbo-0125': 346},5,0,10.444064,-4162347307662832090
2,gpt-3.5-turbo-0125,10,1.0,{'gpt-3.5-turbo-0125': 1920},{'gpt-3.5-turbo-0125': 340},5,0,9.815209,-2576078277911081312
3,gpt-4-turbo-preview,3,1.0,{'gpt-3.5-turbo-0125': 1913},{'gpt-3.5-turbo-0125': 341},5,0,10.526557,6744964500633032433
4,gpt-4-turbo-preview,5,1.0,{'gpt-3.5-turbo-0125': 1944},{'gpt-3.5-turbo-0125': 360},5,0,11.934766,-2228734790447148756
5,gpt-4-turbo-preview,10,1.0,{'gpt-3.5-turbo-0125': 1920},{'gpt-3.5-turbo-0125': 340},5,0,11.583412,-5529193434461873589


Let's do the same for our heiarchical embedding approach. This time we'll just vary the model selection for each step. 

In [316]:
categorizer_llm_grid = pipeline.Pipeline([
  first_level_category_step, 
  select_first_category_step,
  second_level_category_step,
  select_second_category_step,
  combine_taxonomy_step
], evaluation_fn=evaluate2)


params_grid = {
    'first_level_category_step': {
        'model': [models.gpt35, models.gpt4],  
    },
    'second_level_category_step': {
        'model': [models.gpt35, models.gpt4],  
    },
}

small_df2 = df.head(5).copy()

grid_search2 = GridSearch(categorizer_llm_grid, params_grid)

results_df2 = grid_search.apply(small_df2)

Iteration 1 of 4
Params:  {'first_level_category_step': {'model': 'gpt-3.5-turbo-0125'}, 'second_level_category_step': {'model': 'gpt-3.5-turbo-0125'}}
Result:  {'first_level_category_step__model': 'gpt-3.5-turbo-0125', 'second_level_category_step__model': 'gpt-3.5-turbo-0125', 'score': 1.0, 'input_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 2653}), 'output_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 110}), 'num_success': 5, 'num_failure': 0, 'total_latency': 5.898552541009849, 'index': -3130642377686185456}
Iteration 2 of 4
Params:  {'first_level_category_step': {'model': 'gpt-3.5-turbo-0125'}, 'second_level_category_step': {'model': 'gpt-4-0125-preview'}}
Result:  {'first_level_category_step__model': 'gpt-3.5-turbo-0125', 'second_level_category_step__model': 'gpt-4-0125-preview', 'score': 1.0, 'input_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 2653}), 'output_tokens': defaultdict(<class 'int'>, {'gpt-3.5-turbo-0125': 110}), 'num_success': 

In [315]:
# TODO: Grid search seems broken right now. Update with interpretation once it's fixed.
results_df2

,first_level_category_step__model,second_level_category_step__model,score,input_tokens,output_tokens,num_success,num_failure,total_latency,index
0,gpt-3.5-turbo-0125,gpt-3.5-turbo-0125,0.8,{'gpt-3.5-turbo-0125': 2653},{'gpt-3.5-turbo-0125': 110},5,0,5.695634,-3130642377686185456
1,gpt-3.5-turbo-0125,gpt-4-0125-preview,1.0,{'gpt-3.5-turbo-0125': 2653},{'gpt-3.5-turbo-0125': 110},5,0,6.063714,4138420926190908965
2,gpt-4-0125-preview,gpt-3.5-turbo-0125,0.8,{'gpt-3.5-turbo-0125': 2653},{'gpt-3.5-turbo-0125': 110},5,0,5.778218,8678693043304020716
3,gpt-4-0125-preview,gpt-4-0125-preview,1.0,{'gpt-3.5-turbo-0125': 2653},{'gpt-3.5-turbo-0125': 110},5,0,6.504401,-2420333855811006991
